In [ ]:
import dask.dataframe as dd
from constants import TASK1_OUT_ROOT, RAW_DATA_ROOT, TASK1_NP_SCHEMA, RESULTS_ROOT
import os
from glob import glob
import timeit
import numpy as np
import pandas as pd
from utils import get_total_size_GB

def get_raw_files(root_path, year):
    return sorted(list(glob(f"{root_path}/yellow_tripdata_{year}*.parquet")))


def analize_format(paths, id):
    # read and concat the data (if needed)
    # measure the time needed for reading and concating
    dfs = []
    format = paths[0].split(".")[-1]
    match format:
        case "parquet":
            dfs = [dd.read_parquet(p) for p in paths]
        case "csv":
            # parse dates separately because pandas backend doesn't support dtype=datetime directly during read_csv :(
            datetime_cols = ["pickup_datetime", "dropoff_datetime"]
            corrected_schema = TASK1_NP_SCHEMA.copy()
            del corrected_schema["pickup_datetime"]
            del corrected_schema["dropoff_datetime"]
            dfs = [dd.read_csv(p, dtype=corrected_schema, parse_dates=datetime_cols) for p in paths]
        case "h5":
            dfs = [dd.read_hdf(p, key="taxidata") for p in paths]
            # parse dates separately because pandas backend doesn't support dtype=datetime directly during read_hdf :(
            for i in range(len(dfs)):
                dfs[i]["pickup_datetime"] = dd.to_datetime(dfs[i]["pickup_datetime"])
                dfs[i]["dropoff_datetime"] = dd.to_datetime(dfs[i]["dropoff_datetime"])
    
    # timeit read, concat and compute (multiple runs for average and std)
    times = []
    df = None
    for _ in range(5):
        start = timeit.default_timer()
        df = dd.concat(dfs)
        df = df.compute()
        end = timeit.default_timer()
        times.append(end - start)
    
    # display size of df in memory
    n_rows, n_cols = df.shape
    results = {
        "id": id,
        "format": format,
        "n_rows": n_rows,
        "n_cols": n_cols,
        "mem_usage_GB": df.memory_usage(deep=True).sum() / (1024 * 1024 * 1024),
        "mean_read_time": np.mean(times),
        "std_read_time": np.std(times),
        "size_on_disk_GB": get_total_size_GB(paths),
    }

    del df # force cleanup
    return results

all_original_parquet = list(glob(os.path.join(RAW_DATA_ROOT, "yellow_tripdata_*.parquet")))
five_years_original_parquet = sum([get_raw_files(RAW_DATA_ROOT, year) for year in range(2020, 2025)], [])
one_year_original_parquet = get_raw_files(RAW_DATA_ROOT, 2024)

all_parquet = list(glob(os.path.join(TASK1_OUT_ROOT, "all", "*", "*.parquet")))
five_years_parquet = list(filter(lambda x: any([os.path.dirname(x).endswith(y) for y in ["2020","2021","2022","2023","2024"]]), all_parquet))
five_years_csv = os.path.join(TASK1_OUT_ROOT, "five_years","2020_2024.csv")
one_year_parquet = list(filter(lambda x: os.path.dirname(x).endswith("2024"), all_parquet))
one_year_csv = os.path.join(TASK1_OUT_ROOT, "one_year","2024.csv")
one_year_hdf5 = os.path.join(TASK1_OUT_ROOT, "one_year","2024.h5")

assert len(all_original_parquet) == 193, f"Expected 193 original parquet files, but found {len(all_original_parquet)}"
assert len(all_parquet) == 193, f"Expected 193 parquet files, but found {len(all_parquet)}"
assert len(five_years_original_parquet) == 12*5, f"Expected 60 parquet files for 5 years, but found {len(five_years_original_parquet)}"
assert len(five_years_parquet) == 12*5
assert len(one_year_original_parquet) == 12, f"Expected 12 parquet files for 1 year, but found {len(one_year_original_parquet)}"
assert len(one_year_parquet) == 12


### Sample rows from original files

In [2]:
df_original = dd.concat([dd.read_parquet(p) for p in one_year_original_parquet])
display(df_original.head(10))
display(df_original.dtypes)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.7,1.0,0.5,0.00,0.0,1.0,22.70,2.5,0.00
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.0,3.5,0.5,3.75,0.0,1.0,18.75,2.5,0.00
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.3,3.5,0.5,3.00,0.0,1.0,31.30,2.5,0.00
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.0,3.5,0.5,2.00,0.0,1.0,17.00,2.5,0.00
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.9,3.5,0.5,3.20,0.0,1.0,16.10,2.5,0.00
5,1,2024-01-01 00:54:08,2024-01-01 01:26:31,1.0,4.70,1.0,N,148,141,1,29.6,3.5,0.5,6.90,0.0,1.0,41.50,2.5,0.00
6,2,2024-01-01 00:49:44,2024-01-01 01:15:47,2.0,10.82,1.0,N,138,181,1,45.7,6.0,0.5,10.00,0.0,1.0,64.95,0.0,1.75
7,1,2024-01-01 00:30:40,2024-01-01 00:58:40,0.0,3.00,1.0,N,246,231,2,25.4,3.5,0.5,0.00,0.0,1.0,30.40,2.5,0.00
8,2,2024-01-01 00:26:01,2024-01-01 00:54:12,1.0,5.44,1.0,N,161,261,2,31.0,1.0,0.5,0.00,0.0,1.0,36.00,2.5,0.00
9,2,2024-01-01 00:28:08,2024-01-01 00:29:16,1.0,0.04,1.0,N,113,113,2,3.0,1.0,0.5,0.00,0.0,1.0,8.00,2.5,0.00


VendorID                           int32
tpep_pickup_datetime      datetime64[us]
tpep_dropoff_datetime     datetime64[us]
passenger_count                    int64
trip_distance                    float64
RatecodeID                         int64
store_and_fwd_flag       string[pyarrow]
PULocationID                       int32
DOLocationID                       int32
payment_type                       int64
fare_amount                      float64
extra                            float64
mta_tax                          float64
tip_amount                       float64
tolls_amount                     float64
improvement_surcharge            float64
total_amount                     float64
congestion_surcharge             float64
Airport_fee                      float64
dtype: object

### Sample rows from the processed dataset

In [3]:
df_processed = dd.concat([dd.read_parquet(p) for p in one_year_parquet])
display(df_processed.head(10))
display(df_processed.dtypes)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,rate_code_id,store_and_fwd_flag,payment_type,fare_amount,extra,...,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,year
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1,1.72,1,0,2,17.700001,1.0,...,0.0,1.0,22.700001,2.5,0.00,-73.992439,40.748497,-73.985939,40.727619,2024
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1,1.80,1,0,1,10.000000,3.5,...,0.0,1.0,18.750000,2.5,0.00,-73.954742,40.765484,-73.957008,40.780437,2024
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1,4.70,1,0,1,23.299999,3.5,...,0.0,1.0,31.299999,2.5,0.00,-73.957008,40.780437,-73.985939,40.727619,2024
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1,1.40,1,0,1,10.000000,3.5,...,0.0,1.0,17.000000,2.5,0.00,-73.985939,40.727619,-74.001541,40.723888,2024
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1,0.80,1,0,1,7.900000,3.5,...,0.0,1.0,16.100000,2.5,0.00,-74.001541,40.723888,-73.990898,40.718937,2024
5,1,2024-01-01 00:54:08,2024-01-01 01:26:31,1,4.70,1,0,1,29.600000,3.5,...,0.0,1.0,41.500000,2.5,0.00,-73.990898,40.718937,-73.959633,40.766949,2024
6,2,2024-01-01 00:49:44,2024-01-01 01:15:47,2,10.82,1,0,1,45.700001,6.0,...,0.0,1.0,64.949997,0.0,1.75,-73.873627,40.774376,-73.981415,40.670376,2024
7,1,2024-01-01 00:30:40,2024-01-01 00:58:40,0,3.00,1,0,2,25.400000,3.5,...,0.0,1.0,30.400000,2.5,0.00,-74.004013,40.753307,-74.007881,40.717773,2024
8,2,2024-01-01 00:26:01,2024-01-01 00:54:12,1,5.44,1,0,2,31.000000,1.0,...,0.0,1.0,36.000000,2.5,0.00,-73.977699,40.758026,-74.013023,40.709141,2024
9,2,2024-01-01 00:28:08,2024-01-01 00:29:16,1,0.04,1,0,2,3.000000,1.0,...,0.0,1.0,8.000000,2.5,0.00,-73.994308,40.732578,-73.994308,40.732578,2024


vendor_id                          int8
pickup_datetime          datetime64[ns]
dropoff_datetime         datetime64[ns]
passenger_count                   uint8
trip_distance                   float32
rate_code_id                      uint8
store_and_fwd_flag                 int8
payment_type                      uint8
fare_amount                     float32
extra                           float32
mta_tax                         float32
tip_amount                      float32
tolls_amount                    float32
improvement_surcharge           float32
total_amount                    float32
congestion_surcharge            float32
airport_fee                     float32
pickup_longitude                float32
pickup_latitude                 float32
dropoff_longitude               float32
dropoff_latitude                float32
year                           category
dtype: object

### File sizes

In [2]:
print("Original data sizes:")
print(f"Total size (all years, parquet): {get_total_size_GB(all_original_parquet)} GB")
print(f"Total size (2020-2024, parquet): {get_total_size_GB(five_years_original_parquet)} GB")
print(f"Total size (2024, parquet):      {get_total_size_GB(one_year_original_parquet)} GB")

print("Processed data sizes:")
print(f"Total size (all years, parquet): {get_total_size_GB(all_parquet)} GB")
print(f"Total size (2020-2024, parquet): {get_total_size_GB(five_years_parquet)} GB")
print(f"Total size (2020-2024, csv):     {get_total_size_GB([five_years_csv])} GB")
print(f"Total size (2024, parquet):      {get_total_size_GB(one_year_parquet)} GB")
print(f"Total size (2024, csv):          {get_total_size_GB([one_year_csv])} GB")
print(f"Total size (2024, hdf5):         {get_total_size_GB([one_year_hdf5])} GB")

Original data sizes:
Total size (all years, parquet): 28.941 GB
Total size (2020-2024, parquet): 2.606 GB
Total size (2024, parquet):      0.645 GB
Processed data sizes:
Total size (all years, parquet): 35.057 GB
Total size (2020-2024, parquet): 3.772 GB
Total size (2020-2024, csv):     22.429 GB
Total size (2024, parquet):      0.895 GB
Total size (2024, csv):          5.305 GB
Total size (2024, hdf5):         1.014 GB


### Comments
- As expected, the binary formats (parquet and HDF) achieve the smallest file-sizes with parquet pulling ahead.
- Our processed datasets are larger than the original parquet ones due to extra columns.

### Comparing formats

In [3]:
format_results = []
format_results.append(analize_format(one_year_original_parquet, "original_1year"))
format_results.append(analize_format(one_year_parquet, "processed_1year"))
format_results.append(analize_format([one_year_csv], "processed_1year"))
format_results.append(analize_format([one_year_hdf5], "processed_1year"))

In [4]:
format_results_df = pd.DataFrame(format_results)
format_results_df["mem_usage_GB"] = format_results_df["mem_usage_GB"].round(3)
format_results_df["mean_read_time"] = format_results_df["mean_read_time"].round(2)
format_results_df["std_read_time"] = format_results_df["std_read_time"].round(2)
format_results_df["size_on_disk_GB"] = format_results_df["size_on_disk_GB"].round(3)
format_results_df.to_csv(os.path.join(RESULTS_ROOT, "format_analysis.csv"), index=False)
display(format_results_df)

,id,format,n_rows,n_cols,mem_usage_GB,mean_read_time,std_read_time,size_on_disk_GB
0,original_1year,parquet,41169720,19,5.714,3.32,0.83,0.645
1,processed_1year,parquet,41169720,22,3.297,4.12,1.75,0.895
2,processed_1year,csv,41169720,22,3.336,89.80,1.87,5.305
3,processed_1year,h5,41169720,22,3.336,16.07,0.32,1.014


### Comments
- The original parquet files have wasteful datatypes - our processed datasets have more columns, yet take up 42% less space in memory. The main culprits are the string datatypes (e.g. store_and_forward flag column is a string of either 'Y' or 'N'. We change this to 1 or 0 respectively).
- Out of all the formats, parquet is 1 to 2 orders of magnitude faster compared to other formats (CSV being the slowest)
- We had trouble using dask's implementation of HDF (dask.dataframe.to_hdf) as it produced larger files than the CSV equivalents. Using a different implementation (h5py) we achieve similar file sizes to parquet.
- Bottom-line: for this project we will stick to parquet as it is the fastest to work with, preserves data-types (CSV does not, while HDF allows only ints/floats so datetimes are not parsed automatically). Additionally, plain-text formats like CSV lose the appeal in this case as we cannot inspect them in a text editor due to the sheer size of the file.